# KPI : Acteurs les plus présents et périodes associées.

- **Objectif** : 

    - obtenir des DataFrames optimisés pour Power BI    

analyse approfondie de votre base de données pour identifier des tendances et caractéristiques spécifiques. Cette analyse devrait inclure : l’identification des acteurs les plus présents et les périodes associées, l’évolution de la durée moyenne des films au fil des années, la comparaison entre les acteurs présents au cinéma et dans les séries, l’âge moyen des acteurs, ainsi que les films les mieux notés et les caractéristiques qu’ils partagent.

In [3]:
import pandas as pd
import numpy as np

## `title_basics`

In [22]:

# Chargement de la table title.basics
file_path = '../../gitignore/title.basics.tsv'
title_basics = pd.read_csv(file_path, sep='\t', na_values='\\N')

# Aperçu rapide des données
title_basics.head()

C:\Users\jpvt\AppData\Local\Temp\ipykernel_12528\3904950657.py:5: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  title_basics = pd.read_csv(file_path, sep='\t', na_values='\\N')


,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0.0,1894.0,NaN,1.0,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0.0,1892.0,NaN,5.0,"Animation,Short"
2,tt0000003,short,Poor Pierrot,Pauvre Pierrot,0.0,1892.0,NaN,5.0,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0.0,1892.0,NaN,12.0,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0.0,1893.0,NaN,1.0,"Comedy,Short"


In [23]:
# Création de la colonne 'support' basée sur la colonne 'titleType'
def classify_support(titleType):
    if titleType in ['movie', 'short']:
        return 'Cinéma'
    elif titleType in ['tvSeries', 'tvMiniSeries', 'tvMovie', 'tvSpecial', 'tvEpisode', 'tvShort', 'tvPilot']:
        return 'Télévision'
    else:
        return 'Autre'

# Application de la fonction
title_basics['support'] = title_basics['titleType'].apply(classify_support)

In [24]:
title_basics['support'].value_counts()

support
Télévision    9180552
Cinéma        1726297
Autre          341250
Name: count, dtype: int64

In [25]:
# Suppression des colonnes inutiles
title_basics = title_basics.drop(columns=['endYear'])

# Renommage des colonnes en français
title_basics = title_basics.rename(columns={
    'tconst': 'tconst',
    'titleType': 'Type',
    'primaryTitle': 'Titre Principal',
    'originalTitle': 'Titre Original',
    'isAdult': 'pour_adultes',
    'startYear': 'Année Sortie',
    'runtimeMinutes': 'Durée',
    'genres': 'Genres',
    'support': 'Support'
})

# Conversion des types de données
title_basics['pour_adultes'] = title_basics['pour_adultes'].astype('bool')

In [26]:
# Suppression des lignes où adulte = True
title_basics = title_basics[title_basics['pour_adultes'] == False].reset_index(drop=True)

In [27]:
# Suppression de la colonne Adulte
title_basics = title_basics.drop(columns=['pour_adultes'])

In [28]:
title_basics.head()

,tconst,Type,Titre Principal,Titre Original,Année Sortie,Durée,Genres,Support
0,tt0000001,short,Carmencita,Carmencita,1894.0,1.0,"Documentary,Short",Cinéma
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,1892.0,5.0,"Animation,Short",Cinéma
2,tt0000003,short,Poor Pierrot,Pauvre Pierrot,1892.0,5.0,"Animation,Comedy,Romance",Cinéma
3,tt0000004,short,Un bon bock,Un bon bock,1892.0,12.0,"Animation,Short",Cinéma
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,1893.0,1.0,"Comedy,Short",Cinéma


In [29]:
title_basics['tconst'].duplicated().sum()

0

In [30]:
title_basics = title_basics.replace('\\N', np.nan)

In [32]:
title_basics.head()

,tconst,Type,Titre Principal,Titre Original,Année Sortie,Durée,Genres,Support
0,tt0000001,short,Carmencita,Carmencita,1894.0,1.0,"Documentary,Short",Cinéma
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,1892.0,5.0,"Animation,Short",Cinéma
2,tt0000003,short,Poor Pierrot,Pauvre Pierrot,1892.0,5.0,"Animation,Comedy,Romance",Cinéma
3,tt0000004,short,Un bon bock,Un bon bock,1892.0,12.0,"Animation,Short",Cinéma
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,1893.0,1.0,"Comedy,Short",Cinéma


In [33]:
# Export temporaire en cas de crash
title_basics.to_csv('../../gitignore/title_basics_kpi1.csv', index=False)

## `title_principals`

In [34]:
# Chargement de la table title.principals
file_path = '../../gitignore/title.principals.tsv'
title_principals = pd.read_csv(file_path, sep='\t', na_values='\\N')

# Aperçu rapide des données
title_principals.head()

,tconst,ordering,nconst,category,job,characters
0,tt0000001,1,nm1588970,self,NaN,"[""Self""]"
1,tt0000001,2,nm0005690,director,NaN,NaN
2,tt0000001,3,nm0005690,producer,producer,NaN
3,tt0000001,4,nm0374658,cinematographer,director of photography,NaN
4,tt0000002,1,nm0721526,director,NaN,NaN


In [35]:
title_principals['category'].value_counts()

category
actor                  21335705
actress                15996680
self                   12816065
writer                 10738282
director                7697978
producer                6743121
editor                  4708732
cinematographer         3594914
composer                2910416
production_designer     1078929
casting_director        1049804
archive_footage          536848
archive_sound              8879
Name: count, dtype: int64

In [ ]:
# Étape 1 : Suppression des colonnes inutiles
title_principals = title_principals.drop(columns=['job', 'characters', 'ordering'])

# Étape 2 : Filtrer les lignes avec les catégories pertinentes
categories_pertinentes = ['actor', 'actress', 'self']
title_principals_filtered = title_principals[title_principals['category'].isin(categories_pertinentes)]

In [38]:
title_principals_filtered.head()

,tconst,nconst,category
0,tt0000001,nm1588970,self
13,tt0000005,nm0443482,actor
14,tt0000005,nm0653042,actor
16,tt0000007,nm0179163,actor
17,tt0000007,nm0183947,actor


In [40]:
# Export temporaire en cas de crash
title_principals_filtered.to_csv('../../gitignore/title_principals_kpi1.csv', index=False)

In [4]:
import pandas as pd
# Chargement de la table title.principals
file_path = '../../gitignore/title_principals_kpi1.csv'
title_principals = pd.read_csv(file_path, na_values='\\N')

# Aperçu rapide des données
title_principals.head()

,tconst,nconst,category
0,tt0000001,nm1588970,self
1,tt0000005,nm0443482,actor
2,tt0000005,nm0653042,actor
3,tt0000007,nm0179163,actor
4,tt0000007,nm0183947,actor


In [6]:
title_principals_filtered = title_principals.copy()

#### Remplacer `self` par `actor` ou `actress`

- On conserve acteurs et actrices pour analyse par sexe

In [ ]:
# Étape 1 : Identifier les nconst ayant la catégorie 'actor' ou 'actress'
actor_nconst = title_principals_filtered[title_principals_filtered['category'] == 'actor']['nconst'].unique()
actress_nconst = title_principals_filtered[title_principals_filtered['category'] == 'actress']['nconst'].unique()

# Étape 2 : Remplacer 'self' par 'actor' ou 'actress' selon le cas
def update_category(row):
    if row['category'] == 'self':
        if row['nconst'] in actor_nconst:
            return 'actor'
        elif row['nconst'] in actress_nconst:
            return 'actress'
    return row['category']

# Appliquer la fonction pour mettre à jour la colonne 'category'
title_principals_filtered['category'] = title_principals_filtered.apply(update_category, axis=1)

In [ ]:
# Export temporaire en cas de crash
title_principals_filtered.to_csv('../../gitignore/title_principals_kpi2.csv', index=False)

In [2]:
import pandas as pd

title_principals_filtered =pd.read_csv('../../gitignore/title_principals_kpi1.csv')

In [4]:
title_principals_filtered.sample(10)

,tconst,nconst,category
32256064,tt29715007,nm5908207,actor
34757835,tt32311019,nm1402088,actress
1513064,tt0203423,nm0849433,actor
30810884,tt28477114,nm15075367,actress
47114672,tt8510604,nm1739997,self
35370616,tt3278264,nm3866213,actor
10098851,tt1131208,nm0700175,actor
5875678,tt0866151,nm0344188,actor
14875183,tt13462154,nm0741299,self
40620882,tt5370346,nm0056030,actor


In [8]:
# Grouper par tconst et concaténer les nconst par catégorie
df_grouped = title_principals_filtered.groupby(["tconst", "category"])["nconst"].apply(lambda x: ", ".join(x)).reset_index()

In [10]:
# Réorganiser pour avoir une seule ligne par tconst
df_final = df_grouped.pivot(index="tconst", columns="category", values="nconst").reset_index()

In [11]:
df_final.to_csv('../../gitignore/title_principals_kpi3.csv', index=False)

In [13]:
df_final['tconst'].duplicated().sum()

0

## `title.akas`

In [4]:
df_akas = pd.read_csv('../../gitignore/title.akas.tsv', sep='\t', low_memory=False)

In [1]:
import pandas as pd

info_equipe = pd.read_csv('../../gitignore/info_équipe_film.tsv', sep='\t')

In [5]:
info_equipe.head()

,tconst,nconst,catégorie
0,tt0000001,nm0005690,director
1,tt0000001,nm0005690,producer
2,tt0000001,nm0374658,cinematographer
3,tt0000002,nm0721526,director
4,tt0000002,nm1335271,composer


In [4]:
info_equipe = info_equipe.drop(columns='poste')

In [6]:
# Grouper par tconst et concaténer les nconst par catégorie
df_grouped = info_equipe.groupby(["tconst", "catégorie"])["nconst"].apply(lambda x: ", ".join(x)).reset_index()

In [7]:
# Réorganiser pour avoir une seule ligne par tconst
df_final = df_grouped.pivot(index="tconst", columns="catégorie", values="nconst").reset_index()

In [9]:
df_final['tconst'].duplicated().sum()

0

In [10]:
df_final.to_csv('../../gitignore/info_equipe_film_v2.csv', index=False)